# Happy or not?

I'll be using transfer learning with keras, to find out if a certain image depicts a happy person or not. There might be better or more efficient approaches to doing this, and maybe this turns out to be overkill. However, I wish to know if this would work well. 
I'll be using pre-trained inception model v3 and add a couple of FC layers at the end to check how well it does. 
For this I have a dataset of about 250 images scrapped off the internet. I would suggest you to not use google since it is biased in the sense you would not get asians or indians pictures in the pool. Bing is better. Best would be instagram. 

In [1]:
import numpy as np
import pandas as pd
import h5py

import os, cv2, random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from matplotlib import ticker
%matplotlib inline 

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, GlobalAveragePooling2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

# First attempt will be with the inception v3 model
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import backend as K

Using TensorFlow backend.
/home/paperspace/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
TRAIN_DIR = './dataset/'
CLASSES = ['happy', 'nothappy']

# Image dimentions for resizing:
ROWS = 299  # Default input of inception v3 
COLS = 299 # Default input of inception v3 
CHANNELS = 3

In [33]:
def get_images(emotion):
    """Load files from train folder"""
    emotion_dir = TRAIN_DIR+'{}'.format(emotion)
    images = [emotion+'/'+im for im in os.listdir(emotion_dir)]
    return images

def read_image(src):
    """Read and resize individual images"""
    im = cv2.imread(src, cv2.IMREAD_COLOR)
    im = cv2.resize(im, (COLS, ROWS), interpolation=cv2.INTER_CUBIC)
    return im


files = []
y_all = []

for emotion in CLASSES:
    emotion_files = get_images(emotion)
    files.extend(emotion_files)
    
    y_emotion = np.tile(emotion, len(emotion_files))
    y_all.extend(y_emotion)
    print("{0} photos of {1}".format(len(emotion_files), emotion))
    
y_all = np.array(y_all)

256 photos of happy
190 photos of nothappy


In [34]:
X_all = np.ndarray((len(files), ROWS, COLS, CHANNELS), dtype=np.uint8)

In [35]:
for i, im in enumerate(files): 
    X_all[i] = read_image(TRAIN_DIR+im)
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))

print(X_all.shape)

Processed 0 of 446
(446, 299, 299, 3)


In [36]:
y_all

array(['happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'hap

In [37]:
y_all = LabelEncoder().fit_transform(y_all)
y_all.shape

(446,)

In [38]:
y_all[0]

0

In [39]:
y_all_cat = np_utils.to_categorical(y_all)
y_all_cat[0]

array([ 1.,  0.])

In [40]:
X_train, X_valid, y_train, y_valid = train_test_split(X_all, y_all, 
                                                    test_size=0.2, random_state=23, 
                                                    stratify=y_all)

In [41]:
X_train.shape

(356, 299, 299, 3)

In [42]:
y_train.shape

(356,)

In [43]:
X_valid.shape

(90, 299, 299, 3)

In [14]:
# Remember that as a part of the preprocessing we have already scaled down the images to fit
# the inception model input requirements.

# create the base pre-trained model from Keras library
base_model = InceptionV3(weights='imagenet', include_top=False) #change weights to 'imagenet' on your local build

In [15]:
import tensorflow as tf
print(tf.__version__)

1.4.0


In [16]:
#####################
# USING SOFTMAX
#####################

x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# and a logistic layer for the FISH_CLASSES = 8 we are trying to predict
predictions = Dense(len(CLASSES), activation='softmax', name='prediction_layer')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)


# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first N layers and unfreeze the rest:

N=172

for layer in model.layers[:N]:
    layer.trainable = False
for layer in model.layers[N:]:
    layer.trainable = True
    
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [44]:
####################
# USING SIGMOID
####################

x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)

predictions = Dense(1, activation='sigmoid', name='prediction_layer')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first N layers and unfreeze the rest:
N=172

for layer in model.layers[:N]:
    layer.trainable = False
for layer in model.layers[N:]:
    layer.trainable = True
    
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

In [45]:
len(model.layers)

315

In [46]:
model.fit(X_train, y_train, batch_size=20, epochs=20,
              validation_split=0.2, verbose=1, shuffle=True)

Train on 284 samples, validate on 72 samples
Epoch 1/20
284/284 [==============================] - 13s - loss: 0.7165 - acc: 0.4542 - val_loss: 0.7079 - val_acc: 0.4444
Epoch 2/20
284/284 [==============================] - 7s - loss: 0.6603 - acc: 0.6197 - val_loss: 0.6519 - val_acc: 0.7222
Epoch 3/20
284/284 [==============================] - 7s - loss: 0.6377 - acc: 0.6866 - val_loss: 0.6220 - val_acc: 0.6806
Epoch 4/20
284/284 [==============================] - 7s - loss: 0.6080 - acc: 0.7465 - val_loss: 0.6036 - val_acc: 0.6944
Epoch 5/20
284/284 [==============================] - 7s - loss: 0.5848 - acc: 0.8169 - val_loss: 0.5914 - val_acc: 0.7361
Epoch 6/20
284/284 [==============================] - 7s - loss: 0.5696 - acc: 0.8134 - val_loss: 0.5791 - val_acc: 0.7917
Epoch 7/20
284/284 [==============================] - 7s - loss: 0.5530 - acc: 0.8415 - val_loss: 0.5631 - val_acc: 0.7917
Epoch 8/20
284/284 [==============================] - 7s - loss: 0.5337 - acc: 0.8627 - val_l

In [47]:
loss_and_metrics = model.evaluate(X_valid, y_valid, batch_size=20)
print ("Validation Logloss: ", loss_and_metrics)

90/90 [==============================] - 1s     
Validation Logloss:  [0.37530223859681022, 0.93333333068423796]


In [48]:
TEST_DIR = './test/'
os.listdir(TEST_DIR)

['happy-group.jpg',
 'sad-baby-crying.jpg',
 'sad-bearded-man.jpg',
 'happy-lady.jpg',
 'sad-kid-crying.jpg',
 'o-david-cameron-sad-facebook.jpg',
 'sad-baby-heartbroken.jpg']

In [49]:

test_files = [im for im in os.listdir(TEST_DIR) if im.endswith('jpg')]
test = np.ndarray((len(test_files), ROWS, COLS, CHANNELS), dtype=np.uint8)

for i, im in enumerate(test_files): 
    test[i] = read_image(TEST_DIR+im)

test.shape

(7, 299, 299, 3)

In [50]:
test_preds = model.predict(test, verbose=1)

7/7 [==============================] - 1s


In [51]:
test_preds

array([[ 0.27315632],
       [ 0.30323479],
       [ 0.6812585 ],
       [ 0.3893396 ],
       [ 0.66151845],
       [ 0.34654906],
       [ 0.46705243]], dtype=float32)

In [53]:
submission = pd.DataFrame(test_preds, columns=['happy?'])
submission.insert(0, 'image', test_files)
submission.head()

,image,happy?
0,happy-group.jpg,0.273156
1,sad-baby-crying.jpg,0.303235
2,sad-bearded-man.jpg,0.681258
3,happy-lady.jpg,0.389340
4,sad-kid-crying.jpg,0.661518


So, as we see, the results are good. They would be even better if we train for more epochs. :-)

**Let's save the model, so we could deploy the current version**

In [54]:
model.save('../models/happy-or-not-v2-binary.h5')